#### 4 segment files received after cut work for Wiki and News document
#### 维基百科和新闻文件分词过后一共得到4个文件： segment_0 to segment_3

In [1]:
!ls '../segment'

ls: ../segment: No such file or directory


In [1]:
# combined the 4 segment txt file to one to get word count
#!cat segment_0.txt segment_1.txt segment_2.txt segment_3.txt > segment_combined.txt
!ls '../segment'

segment_0.txt        segment_2.txt        segment_combined.txt
segment_1.txt        segment_3.txt


In [3]:
import re
from collections import Counter
## clean up the segement text before count the frequency

def get_chinese_number_letters(train_file):
    train = train_file
    train = open(train,encoding="utf8", errors='ignore').read().lower()
    #[\u00C0-\u017F] Unicode range for all Latin characters
    #[\u4e00-\u9fff] Unicode range for all chinese characters
    chinese_only=re.findall(r'([\u4e00-\u9fff]+|[A-Z]+|[0-9]+|[a-z]+| [ ])',train)
    
    #chinese_only=re.findall('[\w\s]',train)
    print ('First 20 characters of the train file is :', ' '.join(chinese_only)[:20])
    return ' '.join(chinese_only)

def get_1_gram_count(text):
    print ('Length of cleaned train file: ', len(text))
    TOKENS = text.split(' ')
    print ('Train file contains ', len(TOKENS), ' of words(tokens).')
    _1_gram_word_counts = Counter(TOKENS)
    return _1_gram_word_counts

In [4]:
import time
start_time = time.time()
train_file='../segment/segment_combined.txt'
clean_file=get_chinese_number_letters(train_file)
_1_gram_word_counts=get_1_gram_count(clean_file)
print ("My program took", time.time() - start_time, "to run")

First 20 characters of the train file is : 小米 miui    9 首批 机型 曝
Length of cleaned train file:  676727590
Train file contains  236972239  of words(tokens).
My program took 379.3780951499939 to run


In [24]:
clean_file[:100]

'小米 miui    9 首批 机型 曝光 共计 15 款    此外 自 本周 6 月 12 日 起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验'

In [6]:
import pandas as pd
wordfreq=dict(_1_gram_word_counts)
word_count_df=pd.DataFrame(list(wordfreq.items()),columns=['word', 'freq'])
word_count_df.head()

,word,freq
0,小米,1801
1,miui,125
2,,10129212
3,9,261827
4,首批,6634


In [7]:
#remove the count for space
word_count_df=word_count_df[word_count_df['word']!=''].sort_values('freq',ascending=False)

In [8]:
word_count_df.head()

,word,freq
53,的,11129781
839,年,3304023
75,在,3246309
49,是,2413304
111,和,1698098


### export the full word_count result to word_count.txt

In [9]:
word_count_df.to_csv('../models/word_count.txt', index=False, header=False,sep=' ')

In [10]:
word_count_df.describe()

,freq
count,2.695633e+06
mean,8.415204e+01
std,8.252787e+03
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,4.000000e+00
max,1.112978e+07


### Wiki and News contains 2,695,633 unique words

In [12]:
len(word_count_df)

2695633

### 50% of the word only occurs once

In [13]:
len(word_count_df[word_count_df.freq==1])/2695633.

0.5005714798713327

### 14% of the word occurs twice

In [14]:
len(word_count_df[word_count_df.freq==2])/2695633.

0.14491327268956863

### 7% of the words occurs three times

In [15]:
len(word_count_df[word_count_df.freq==3])/2695633.

0.06798959650664611

### 78% of the word occurs less than 5 times

In [16]:
len(word_count_df[word_count_df.freq<=5])/2695633.

0.7816093659633934

### 85% of the word occurs less than 10 times

In [18]:
len(word_count_df[word_count_df.freq<=10])/2695633.

0.8504373555302224

### By checking some examples of words with freq ==5 or ==10, most of them are names or words that not commonly used 
### Therefore, export a subset of words (freq>10) in to word_count_small.txt files for a simple version of models

In [19]:
word_count_df[word_count_df.freq==5].head(10)

,word,freq
2111447,弥作,5
1173643,人订,5
1173877,wethouders,5
374440,领收,5
152212,孙树忠,5
1945613,莫姆,5
2121747,嘎布津托,5
1977660,牛惠兰,5
1393483,独游于,5
315502,子之位,5


In [21]:
word_count_df[word_count_df.freq==10].head(20)

,word,freq
1402148,贝田,10
850584,dororo,10
1753190,张贵林,10
1492821,教佳,10
197556,邀请书,10
197347,用肥,10
476107,认明,10
435252,李到,10
2476345,徐元白,10
478376,吴庆彤,10


In [22]:
word_count_df[word_count_df.freq>10].to_csv('../models/word_count_small.txt', index=False, header=False, sep=' ')

In [26]:
import pandas as pd

In [37]:
?pd.read_csv()

In [39]:
word_count_small=pd.read_csv('../models/word_count_small.txt', sep=' ',names=['word', 'count'], header=None)

In [40]:
word_count_small.tail()

,word,count
403161,柳诒,11
403162,村亦于,11
403163,仙灵岛,11
403164,以模,11
403165,之辰,11


In [41]:
word_count_small.head()

,word,count
0,的,11129781
1,年,3304023
2,在,3246309
3,是,2413304
4,和,1698098


In [45]:
word_count_small['number_of_words']=word_count_small.apply(lambda x: len(str(x.word)), axis=1)

In [44]:
word_count_small.iloc[8584]

word      NaN
count    2742
Name: 8584, dtype: object

In [46]:
word_count_small['number_of_words'].describe()

count    403166.000000
mean          2.857039
std           1.477422
min           1.000000
25%           2.000000
50%           2.000000
75%           3.000000
max          29.000000
Name: number_of_words, dtype: float64

In [48]:
len(word_count_small[word_count_small.number_of_words==1])/403166.

0.02029685042885561

In [50]:
len(word_count_small[word_count_small.number_of_words==1])

8183